In [ ]:
import scipp as sc
from ess import loki, sans
from ess.logging import configure_workflow
import scippneutron as scn
import plopp as pp

In [ ]:
logger = configure_workflow('sans2d_I_of_Q', filename='sans2d.log')

In [ ]:
# Include effects of gravity?
gravity = True

# Wavelength binning
wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom')
wavelength_bands = sc.concat(
            [wavelength_bins.min(), wavelength_bins.max()], dim='wavelength')

# Q binning
q_bins = sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom')

In [ ]:
# Using only one-fourth of the full spectra 245760 (reserved for first detector)
spectrum_size =  245760//4

# Sample measurement
sample = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.nxs'),
                             spectrum_size=spectrum_size)
# Direct measurement is with the empty sample holder/cuvette
direct = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063091.nxs'),
                             spectrum_size=spectrum_size)

In [ ]:
data_graph, monitor_graph = sans.i_of_q.make_coordinate_transform_graphs(gravity=gravity)

In [ ]:
monitors = {
    'sample': {'incident': sample.attrs["monitor2"].value,
               'transmission': sample.attrs["monitor4"].value},
    'direct': {'incident': direct.attrs["monitor2"].value,
               'transmission': direct.attrs["monitor4"].value}
}

In [ ]:
# Custom SANS2D position offsets
sample_pos_z_offset = 0.053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m
# There is some uncertainity here
monitor4_pos_z_offset = -6.719 * sc.units.m

# Geometry transformation based on the found beam center position 
# TODO: We need to determine this
x_offset = -0.09288 * sc.units.m
y_offset = 0.08195 * sc.units.m
#Testing with not too far off
#x_offset = -0.092 * sc.units.m
#y_offset = 0.083 * sc.units.m

In [ ]:
# Load direct beam function for main detector
direct_beam = loki.io.load_rkh_wav(loki.data.get_path('DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'))
direct_beam = sans.i_of_q.resample_direct_beam(
direct_beam=direct_beam,
wavelength_bins=wavelength_bins)

Will have start looping somewhere here

In [ ]:
beam_positions = [(-0.092, 0.083), (-0.09288,0.08195)]

In [ ]:

for position in beam_positions:
    print('Starting iteration')
    sample_iter = sample.copy(deep=True)
    
    x_offset = position[0] * sc.units.m
    y_offset = position[1] * sc.units.m
    
    # Add pixel shapes
    sample_iter.coords["pixel_width"] = 0.0035 * sc.units.m
    sample_iter.coords["pixel_height"] = 0.002033984375 * sc.units.m

    # Change sample position
    sample_iter.coords["sample_position"].fields.z += sample_pos_z_offset
    # Apply bench offset to pixel positions
    sample_iter.coords["position"].fields.y += bench_pos_y_offset
    # Now shift pixels positions to get the correct beam center
    sample_iter.coords['position'].fields.x += x_offset
    sample_iter.coords['position'].fields.y += y_offset

    # Change transmission monitor position
    monitors['sample']['transmission'].coords["position"].fields.z += monitor4_pos_z_offset
    monitors['direct']['transmission'].coords["position"].fields.z += monitor4_pos_z_offset
    
    #This part is bit problematic - we can do edges but center will be completely off
    #However the idea is that we are not too far off. 
    mask_edges = (
        (sc.abs(sample_iter.coords['position'].fields.x - x_offset) > sc.scalar(0.48, unit='m')) |
        (sc.abs(sample_iter.coords['position'].fields.y - y_offset) > sc.scalar(0.45, unit='m')))

    mask_center = sc.sqrt(
        sample_iter.coords['position'].fields.x**2 +
        sample_iter.coords['position'].fields.y**2) < sc.scalar(0.04, unit='m')

    sample_iter.masks['edges'] = mask_edges
    sample_iter.masks['center'] = mask_center
    
    mask_tof_min = sc.scalar(13000.0, unit='us')
    mask_tof_max = sc.scalar(15750.0, unit='us')
    tof_masked_region = sc.concat([sample_iter.coords['tof']['tof', 0],
                                   mask_tof_min, mask_tof_max,
                                   sample_iter.coords['tof']['tof', -1]], dim='tof')

    sample_iter = sample_iter.bin(tof=tof_masked_region)
    sample_iter.masks['bragg_peaks'] = sc.array(dims=['tof'], values=[False, True, False])
    
    mask_up = (
       sample_iter.coords['position'].fields.y - y_offset > sc.scalar(-y_offset.value, unit='m')) 
    mask_down = (
       sample_iter.coords['position'].fields.y - y_offset < sc.scalar(-y_offset.value, unit='m')) 
    mask_left = (
       sample_iter.coords['position'].fields.x - x_offset < sc.scalar(-x_offset.value, unit='m')) 
    mask_right = (
       sample_iter.coords['position'].fields.x - x_offset > sc.scalar(-x_offset.value, unit='m')) 
    
    sample_up = sample_iter.copy()
    sample_down = sample_iter.copy()
    sample_left = sample_iter.copy()
    sample_right = sample_iter.copy()
    
    sample_up.masks['mask_down'] = mask_down
    sample_down.masks['mask_up'] = mask_up
    sample_left.masks['mask_right'] = mask_right
    sample_right.masks['mask_left'] = mask_left
    
    #scn.instrument_view(sample_up, pixel_size=0.0075)
    #scn.instrument_view(sample_down, pixel_size=0.0075)
    
    sample_iter, monitors_iter = sans.i_of_q.convert_to_wavelength(
    data=sample_iter,
    monitors=monitors,
    data_graph=data_graph,
    monitor_graph=monitor_graph)
    
    non_background_range = sc.array(dims=['wavelength'], values=[0.7, 17.1], unit='angstrom')
    
    monitors_iter = sans.i_of_q.denoise_and_rebin_monitors(
    monitors=monitors_iter,
    wavelength_bins=wavelength_bins,
    non_background_range=non_background_range)
    
    transmission_fraction = sans.normalization.transmission_fraction(
    data_monitors=monitors_iter['sample'], direct_monitors=monitors_iter['direct'])
    transmission_fraction

    
    solid_angle = sans.normalization.solid_angle_of_rectangular_pixels(
    sample_iter,
    pixel_width=sample_iter.coords['pixel_width'],
    pixel_height=sample_iter.coords['pixel_height'])
    
    quadrants = []
    for sample in [sample_up, sample_down, sample_left, sample_right]:

        sample, monitors = sans.i_of_q.convert_to_wavelength(
        data=sample,
        monitors=monitors,
        data_graph=data_graph,
        monitor_graph=monitor_graph)

        sample_q = sans.i_of_q.convert_to_q_and_merge_spectra(
        data=sample,
        graph=data_graph,
        wavelength_bands=wavelength_bands,
        q_bins=q_bins,
        gravity=gravity)

        quadrants.append(sample_q)
        
    denominator = sans.normalization.compute_denominator(
        direct_beam=direct_beam,
        data_incident_monitor=monitors_iter['sample']['incident'],
        transmission_fraction=transmission_fraction,
        solid_angle=solid_angle)
    
    for coord in ['position', 'sample_position', 'source_position']:
        denominator.coords[coord] = sample_iter.meta[coord]
     
    denominator_q = sans.i_of_q.convert_to_q_and_merge_spectra(
        data=denominator,
        graph=data_graph,
        wavelength_bands=wavelength_bands,
        q_bins=q_bins,
        gravity=True)
    
    
    sample_normalized_up = sans.normalization.normalize(
            numerator=quadrants[0],
            denominator=denominator_q)

    sample_normalized_down = sans.normalization.normalize(
            numerator=quadrants[1],
            denominator=denominator_q)

    sample_normalized_left = sans.normalization.normalize(
            numerator=quadrants[2],
            denominator=denominator_q)

    sample_normalized_right = sans.normalization.normalize(
            numerator=quadrants[3],
            denominator=denominator_q)
    
    sc.plot({'up':sample_normalized_up,'down':sample_normalized_down,
             'left':sample_normalized_left , 'right':sample_normalized_right})

Here we start sample dependent part